In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

spark = SparkSession.builder.master("local[4]").appName("LeadLag").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/12 15:52:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
product_data = [
            (1,"iphone","01-01-2023",1500000),
            (2,"samsung","01-01-2023",1100000),
            (3,"oneplus","01-01-2023",1100000),
            (1,"iphone","01-02-2023",1300000),
            (2,"samsung","01-02-2023",1120000),
            (3,"oneplus","01-02-2023",1120000),
            (1,"iphone","01-03-2023",1600000),
            (2,"samsung","01-03-2023",1080000),
            (3,"oneplus","01-03-2023",1160000),
            (1,"iphone","01-04-2023",1700000),
            (2,"samsung","01-04-2023",1800000),
            (3,"oneplus","01-04-2023",1170000),
            (1,"iphone","01-05-2023",1200000),
            (2,"samsung","01-05-2023",980000),
            (3,"oneplus","01-05-2023",1175000),
            (1,"iphone","01-06-2023",1100000),
            (2,"samsung","01-06-2023",1100000),
            (3,"oneplus","01-06-2023",1200000)
            ]

product_schema = ['product_id', 'product_name', 'sales_date', 'sales']

product_df = spark.createDataFrame(product_data, product_schema)
product_df = product_df.withColumn("sales_date", to_date(col("sales_date"), "dd-MM-yyyy"))
product_df.show()

+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         1|      iphone|2023-01-01|1500000|
|         2|     samsung|2023-01-01|1100000|
|         3|     oneplus|2023-01-01|1100000|
|         1|      iphone|2023-02-01|1300000|
|         2|     samsung|2023-02-01|1120000|
|         3|     oneplus|2023-02-01|1120000|
|         1|      iphone|2023-03-01|1600000|
|         2|     samsung|2023-03-01|1080000|
|         3|     oneplus|2023-03-01|1160000|
|         1|      iphone|2023-04-01|1700000|
|         2|     samsung|2023-04-01|1800000|
|         3|     oneplus|2023-04-01|1170000|
|         1|      iphone|2023-05-01|1200000|
|         2|     samsung|2023-05-01| 980000|
|         3|     oneplus|2023-05-01|1175000|
|         1|      iphone|2023-06-01|1100000|
|         2|     samsung|2023-06-01|1100000|
|         3|     oneplus|2023-06-01|1200000|
+----------+------------+----------+-------+



In [34]:
# Percentage of Sales Each Month based on last 6 Months sales

window_spec = Window.partitionBy(col("product_name"))

df = product_df.withColumn("Total Sales in 6 Month", sum(col("sales")).over(window_spec) )\
                .withColumn("Percnt this month", col("sales")*100 / col("Total Sales in 6 Month"))
df.show()

+----------+------------+----------+-------+----------------------+------------------+
|product_id|product_name|sales_date|  sales|Total Sales in 6 Month| Percnt this month|
+----------+------------+----------+-------+----------------------+------------------+
|         1|      iphone|2023-01-01|1500000|               8400000|17.857142857142858|
|         1|      iphone|2023-02-01|1300000|               8400000|15.476190476190476|
|         1|      iphone|2023-03-01|1600000|               8400000|19.047619047619047|
|         1|      iphone|2023-04-01|1700000|               8400000|20.238095238095237|
|         1|      iphone|2023-05-01|1200000|               8400000|14.285714285714286|
|         1|      iphone|2023-06-01|1100000|               8400000|13.095238095238095|
|         3|     oneplus|2023-01-01|1100000|               6925000|15.884476534296029|
|         3|     oneplus|2023-02-01|1120000|               6925000|16.173285198555956|
|         3|     oneplus|2023-03-01|1160000

In [40]:

df = product_df.withColumn("sales_date", to_date(col("sales_date"), "dd-MM-yyyy"))
df = df.withColumn("six_months_ago", add_months(col("sales_date"), -6))
# df.show()

window_spec = Window.partitionBy("product_name").orderBy("sales_date")

df = df.withColumn( "total_6_months_sales",  sum( col("sales") ).over(window_spec) ).filter(col("sales_date") > col("six_months_ago"))\
        .withColumn("Percnt this month", col("sales")*100 / col("total_6_months_sales"))
df.show()

+----------+------------+----------+-------+--------------+--------------------+------------------+
|product_id|product_name|sales_date|  sales|six_months_ago|total_6_months_sales| Percnt this month|
+----------+------------+----------+-------+--------------+--------------------+------------------+
|         1|      iphone|2023-01-01|1500000|    2022-07-01|             1500000|             100.0|
|         1|      iphone|2023-02-01|1300000|    2022-08-01|             2800000| 46.42857142857143|
|         1|      iphone|2023-03-01|1600000|    2022-09-01|             4400000| 36.36363636363637|
|         1|      iphone|2023-04-01|1700000|    2022-10-01|             6100000|27.868852459016395|
|         1|      iphone|2023-05-01|1200000|    2022-11-01|             7300000|16.438356164383563|
|         1|      iphone|2023-06-01|1100000|    2022-12-01|             8400000|13.095238095238095|
|         3|     oneplus|2023-01-01|1100000|    2022-07-01|             1100000|             100.0|


In [28]:
# Percentage of Profit/Loss based on Prev Month

window_spec = Window.partitionBy("product_id").orderBy(month(col("sales_date")))

last_month_sales_df = product_df.withColumn("previous_month_sales", lag(col("sales"), 1).over(window_spec) )
df = last_month_sales_df.withColumn("per_loss_gain", round( (col("sales") - col("previous_month_sales") )* 100/ col("previous_month_sales") , 2))
df.show()

+----------+------------+----------+-------+--------------------+-------------+
|product_id|product_name|sales_date|  sales|previous_month_sales|per_loss_gain|
+----------+------------+----------+-------+--------------------+-------------+
|         1|      iphone|2023-01-01|1500000|                NULL|         NULL|
|         1|      iphone|2023-02-01|1300000|             1500000|       -13.33|
|         1|      iphone|2023-03-01|1600000|             1300000|        23.08|
|         1|      iphone|2023-04-01|1700000|             1600000|         6.25|
|         1|      iphone|2023-05-01|1200000|             1700000|       -29.41|
|         1|      iphone|2023-06-01|1100000|             1200000|        -8.33|
|         2|     samsung|2023-01-01|1100000|                NULL|         NULL|
|         2|     samsung|2023-02-01|1120000|             1100000|         1.82|
|         2|     samsung|2023-03-01|1080000|             1120000|        -3.57|
|         2|     samsung|2023-04-01|1800

In [41]:
spark.stop()